<a href="https://colab.research.google.com/github/chajunhyeop/-/blob/main/sex_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pingouin
!pip install scikit_posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from matplotlib import pyplot as plt
from pandas import DataFrame
import numpy as np
import seaborn as sns
import pandas as pd
import math
from scipy import stats
import pingouin as pg
import scipy as sp
import scikit_posthocs
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [35]:
from imblearn.over_sampling import SMOTE

In [23]:
body = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/koreaverage.xlsx")

In [24]:
print(body.columns)

Index(['ⓞ_02_성별', 'ⓞ_06_나이_반올림', 'ⓞ_12_골격근량', 'ⓞ_13_체지방량', 'ⓞ_14_체수분',
       'ⓞ_15_단백질', 'ⓞ_16_무기질', 'ⓞ_17_BMI', 'ⓞ_18_체지방율', 'ⓞ_19_복부지방율',
       ...
       '①_124_무릎아래둘레', '①_125_장딴지둘레', '①_126_종아리최소둘레', '①_127_발목최대둘레',
       '①_128_발목너비', '①_129_발뒤꿈치너비', '①_130_위팔둘레', '①_131_팔꿈치둘레', '①_132_손목둘레',
       '①_133_머리위로뻗은주먹높이'],
      dtype='object', length=148)


In [25]:
#열이름 
num=[]
for i in body:
  num.append(i)
  print(i)

ⓞ_02_성별
ⓞ_06_나이_반올림
ⓞ_12_골격근량
ⓞ_13_체지방량
ⓞ_14_체수분
ⓞ_15_단백질
ⓞ_16_무기질
ⓞ_17_BMI
ⓞ_18_체지방율
ⓞ_19_복부지방율
ⓞ_20_복부지방율평가
ⓞ_21_기초대사량
ⓞ_22_기초대사량평가
ⓞ_23_근육조절
ⓞ_24_지방조절
①_001_오른쪽어깨경사각
①_002_왼쪽어깨경사각
①_003_키
①_004_눈높이
①_005_목뒤높이
①_006_어깨높이
①_007_어깨가쪽높이
①_008_겨드랑높이
①_009_굽힌팔꿈치높이
①_010_허리기준선높이
①_011_엉덩이높이
①_012_주먹높이
①_013_허리높이
①_014_배꼽수준허리높이
①_015_위앞엉덩뼈가시높이
①_016_무릎높이
①_017_샅높이
①_018_가슴너비
①_019_젖가슴너비
①_020_허리너비
①_021_배꼽수준허리너비
①_022_엉덩이너비
①_023_겨드랑두께
①_024_가슴두께
①_025_젖가슴두께
①_026_허리두께
①_027_배꼽수준허리두께
①_028_엉덩이두께
①_029_몸통수직길이
①_030_엉덩이수직길이
①_031_몸무게
①_032_앞중심길이
①_033_배꼽수준앞중심길이
①_034_겨드랑앞벽사이길이
①_035_겨드랑앞접힘사이길이
①_036_젖꼭지사이수평길이_여
①_037_목둘레
①_038_목밑둘레
①_039_가슴둘레
①_040_젖가슴둘레
①_041_젖가슴아래둘레_여
①_042_허리둘레
①_043_배꼽수준허리둘레
①_044_배둘레
①_045_엉덩이둘레
①_046_배돌출점기준엉덩이둘레
①_047_어깨길이
①_048_목뒤등뼈위겨드랑수준길이
①_049_등길이
①_050_배꼽수준등길이
①_051_목뒤오금길이
①_052_총길이
①_053_넙다리직선길이
①_054_어깨사이길이
①_055_어깨가쪽사이길이
①_056_겨드랑뒤벽사이길이
①_057_겨드랑뒤벽접힘사이길이
①_058_목뒤젖꼭지길이
①_059_목뒤젖꼭지허리둘레선길이
①_060_목옆젖꼭지길이_여_
①_061_목옆허리둘레선길이
①_062_목옆어깨뼈아래허리둘레선길이
①_063_위팔길이
①_064_팔길이
①

In [26]:
#남성결측값 변수 제거 
df=body.drop(["①_036_젖꼭지사이수평길이_여","①_038_목밑둘레","①_041_젖가슴아래둘레_여","①_060_목옆젖꼭지길이_여_" ,"①_116_얼굴너비","①_117_아래턱사이너비"], axis=1)

In [29]:
#결측값 제거
df.dropna(axis=0, inplace=True)

In [32]:
#결측값 개수 확인
df.isnull().sum()

ⓞ_02_성별             0
ⓞ_06_나이_반올림         0
ⓞ_12_골격근량           0
ⓞ_13_체지방량           0
ⓞ_14_체수분            0
                   ..
①_129_발뒤꿈치너비        0
①_130_위팔둘레          0
①_131_팔꿈치둘레         0
①_132_손목둘레          0
①_133_머리위로뻗은주먹높이    0
Length: 142, dtype: int64

In [31]:
df["ⓞ_02_성별"]

0       남
1       남
2       남
3       남
4       남
       ..
6415    여
6416    여
6417    여
6418    여
6419    여
Name: ⓞ_02_성별, Length: 6341, dtype: object

In [41]:
#연속형 변수와 범주형 변수 분류해주기
num = df.drop(["ⓞ_02_성별", "ⓞ_20_복부지방율평가", "ⓞ_22_기초대사량평가"], axis=1)
num_list = num.columns
cg = ["ⓞ_20_복부지방율평가", "ⓞ_22_기초대사량평가"]

#df 검색시 index에 list를 넣어주어야 함으로 num_list(dtype: object)를 list로 전환해줬다.
num=[]
for i in num_list:
  num.append(i)
  print(i)

#train test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#표준화, 원핫코딩(더미변수화)
ct = ColumnTransformer([('scaling', StandardScaler(), num), ('onehot', OneHotEncoder(sparse=False), cg)])
ct.fit(X_train)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

#분할
X = df[num+cg]
Y = df["ⓞ_02_성별"]

smote = SMOTE(random_state=0)
X_train, Y_train = smote.fit_resample(X_train, Y_train)


ⓞ_06_나이_반올림
ⓞ_12_골격근량
ⓞ_13_체지방량
ⓞ_14_체수분
ⓞ_15_단백질
ⓞ_16_무기질
ⓞ_17_BMI
ⓞ_18_체지방율
ⓞ_19_복부지방율
ⓞ_21_기초대사량
ⓞ_23_근육조절
ⓞ_24_지방조절
①_001_오른쪽어깨경사각
①_002_왼쪽어깨경사각
①_003_키
①_004_눈높이
①_005_목뒤높이
①_006_어깨높이
①_007_어깨가쪽높이
①_008_겨드랑높이
①_009_굽힌팔꿈치높이
①_010_허리기준선높이
①_011_엉덩이높이
①_012_주먹높이
①_013_허리높이
①_014_배꼽수준허리높이
①_015_위앞엉덩뼈가시높이
①_016_무릎높이
①_017_샅높이
①_018_가슴너비
①_019_젖가슴너비
①_020_허리너비
①_021_배꼽수준허리너비
①_022_엉덩이너비
①_023_겨드랑두께
①_024_가슴두께
①_025_젖가슴두께
①_026_허리두께
①_027_배꼽수준허리두께
①_028_엉덩이두께
①_029_몸통수직길이
①_030_엉덩이수직길이
①_031_몸무게
①_032_앞중심길이
①_033_배꼽수준앞중심길이
①_034_겨드랑앞벽사이길이
①_035_겨드랑앞접힘사이길이
①_037_목둘레
①_039_가슴둘레
①_040_젖가슴둘레
①_042_허리둘레
①_043_배꼽수준허리둘레
①_044_배둘레
①_045_엉덩이둘레
①_046_배돌출점기준엉덩이둘레
①_047_어깨길이
①_048_목뒤등뼈위겨드랑수준길이
①_049_등길이
①_050_배꼽수준등길이
①_051_목뒤오금길이
①_052_총길이
①_053_넙다리직선길이
①_054_어깨사이길이
①_055_어깨가쪽사이길이
①_056_겨드랑뒤벽사이길이
①_057_겨드랑뒤벽접힘사이길이
①_058_목뒤젖꼭지길이
①_059_목뒤젖꼭지허리둘레선길이
①_061_목옆허리둘레선길이
①_062_목옆어깨뼈아래허리둘레선길이
①_063_위팔길이
①_064_팔길이
①_065_팔안쪽길이
①_066_목뒤손목안쪽길이
①_067_겨드랑둘레
①_068_엉덩이옆길이
①_069_다리가쪽길이
①_070_몸통세로둘레
①_071_샅앞뒤길이
①_072_배